In [484]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex
!pip install cvxpy
import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import cvxpy as cp

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(Market Beat or Market Meet)

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [487]:
tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name, header = None, names=['Ticker'])


total_investment = 1000000
start_date = "2023-10-01"
end_date = "2024-09-30"
drop_tickers = []
for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        if ticker.info.get('quoteType')=='NONE':
            # print("phewww", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue
                
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            # print("hmmm", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        history = ticker.history(start = start_date, end = end_date)
        
        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18]
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            # print("sheesh", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]

    except:
            # print("ufff", tick)
            pass
        
tickers_file_original = tickers_file_original.reset_index(drop=True)

In [488]:
# Fetch historical data for both indices
sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)['Close']
tsx60 = yf.Ticker("XIU.TO").history(start=start_date, end=end_date)['Close']

In [489]:
mil = 1000000
sp500val = (mil / sp500.iloc[0]) * sp500.iloc[-1]
sp500val = round(sp500val,2)
sp500val

1338070.87

In [490]:
tsx60val = (mil / tsx60.iloc[0]) * tsx60.iloc[-1]
tsx60val = round(tsx60val,2)
tsx60val

1285843.67

In [491]:
average = (sp500val + tsx60val) / 2
average = round(average,2)
average

1311957.27

In [492]:
stocks = {key: 0 for key in tickers_file_original['Ticker'].values}
current_data = pd.DataFrame()
metrics = pd.DataFrame(index=list(stocks.keys()), columns=['SP-beta','SP-corr','TSX-beta','TSX-corr'])

for i in stocks:
    current_data[i] = yf.Ticker(i).history(start=start_date, end=end_date)["Close"]
    covariance = sp500.cov(current_data[i])
    variance = sp500.var()
    beta = covariance / variance

    correlation = sp500.corr(current_data[i])
    metrics.loc[i,'SP-corr'] = correlation
    metrics.loc[i,'SP-beta'] = beta

    covariance2 = tsx60.cov(current_data[i])
    variance2 = tsx60.var()
    beta2 = covariance / variance

    correlation2 = tsx60.corr(current_data[i])
    metrics.loc[i,'TSX-corr'] = correlation2
    metrics.loc[i,'TSX-beta'] = beta2

    stocks[i] = round(100 * (correlation + beta + beta2 + correlation2),2)
    
metrics

,SP-beta,SP-corr,TSX-beta,TSX-corr
AAPL,0.031744,0.678171,0.031744,0.626302
ABBV,0.035126,0.851557,0.035126,0.879057
ABT,0.009455,0.570702,0.009455,0.637014
ACN,0.00463,0.076545,0.00463,0.160075
AIG,0.01192,0.88176,0.01192,0.860027
AMZN,0.042903,0.924021,0.042903,0.865154
AXP,0.081293,0.970728,0.081293,0.971736
BA,-0.031279,-0.528617,-0.031279,-0.530535
BAC,0.010859,0.96535,0.010859,0.929716
BB.TO,-0.001521,-0.86532,-0.001521,-0.811991


In [493]:
stocks = dict(sorted(stocks.items(), key=lambda item: item[1], reverse=True))
stocks = list(stocks.items())[:24]
stocks = dict(stocks)
stocks

{'LLY': 241.08,
 'BLK': 211.54,
 'AXP': 210.5,
 'RY.TO': 197.22,
 'BK': 196.75,
 'CL': 193.52,
 'CAT': 193.34,
 'BAC': 191.68,
 'TXN': 191.35,
 'C': 188.35,
 'AMZN': 187.5,
 'PG': 184.23,
 'ABBV': 180.09,
 'KO': 180.04,
 'AIG': 176.56,
 'QCOM': 174.33,
 'USB': 168.84,
 'MO': 167.77,
 'PM': 160.04,
 'LMT': 158.41,
 'PYPL': 154.33,
 'UNP': 148.9,
 'PEP': 142.54,
 'MRK': 138.51}

In [494]:
def give_weight(num_stocks):
    stocks_now = list(stocks.keys())[:num_stocks]
    prices = np.array(current_data[stocks_now].iloc[0])
    last_values = np.array(current_data[stocks_now].iloc[-1])
    w = cp.Variable(num_stocks)
    my_value = cp.sum(cp.multiply(last_values, w * 1000000 / prices))
    min_weight = (100 / (2 * num_stocks)) / 100
    objective = cp.Minimize(my_value - average)
    constraints = [
        cp.sum(w) == 1,           
        w >= min_weight,                    
        w <= 0.15                      
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return w.value

In [495]:
def get_value(weights,i):
    last_row = current_data[list(stocks.keys())[:i]].iloc[-1]
    first_row = current_data[list(stocks.keys())[:i]].iloc[0]
    return np.sum(np.multiply(last_row,np.divide(weights*1000000,first_row)))

In [496]:
def stocks_ith_portfolio(numStocks):
    weight_now = give_weight(numStocks)
    curr_val = get_value(weight_now, numStocks)
    return curr_val

In [497]:
diff = sp500val
close = 0
best_weights = []
best = 0
for i in range(12,25):
    curr_val = stocks_ith_portfolio(i)
    print(curr_val)
    if (abs(average - curr_val) < diff):
        diff = abs(curr_val - average)
        best = i
        best_weights = weight_now
        close = curr_val

1459597.9988848558
1438591.6433803276
1416746.79298089
1390313.2960502382
1392071.1700818487
1390743.7126010407
1384694.4153357162
1381906.9509988856
1381327.761949757
1376509.6665275076
1357907.379972125
1311846.4323472325
1291474.7570740813


In [522]:
print(average)
print(diff)
print(close)
print(best_weights)

1311957.27
110.83765276754275
1311846.4323472325
[0.02083333 0.02083333 0.02083333 0.02083333 0.02083333 0.02083333
 0.02083333 0.02083333 0.02083333 0.02083333 0.02083333 0.14999999
 0.02083333 0.02083333 0.02083334 0.02083333 0.02083333 0.02083333
 0.02083333 0.02083333 0.02083333 0.13333333 0.15       0.15      ]


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.